# STICKY REVIEWS
#### Navit Dori Capstone Project

As a frequent online shopper I rely heavily on consumer reviews to help me with my shopping.
I have sometimes detected similariteis between consequtive reviews. The similarities were in both text and score. This phenomenonn raised several intriguing questions: is this a measurable effect? Does it reflect a temporary problem with the probduct or is there something in the first review that compelles others to imitate it? Do these reviews have common features?

In this project I define sticky reviews as reviews that start a time-window of low average score. I use Amazon reviews data to measre how common "sticky" reviews are and to detect their common features. This work concentrates on "sticky" reviews that lower the average score although the same analysis may be applied to uncommonly good reviews that raise the average score. 

A temporal decrease in a product score is naturally very important to the seller, whether it is caused by a problem in the product, a competition attack that attempts to lower the score by posting fake reviews, or a bad review that influences others to imitate it. Speaking for myself, the possibility of reviews influencing following reviewers is interesting as a research in human behavior. I am very curious to find out if there are features or attributes of a reviews that can be shown to cause "stickiness".



## Identifying the "sticky" reviews

I used [Amazon reviews data of electronic products](http://jmcauley.ucsd.edu/data/amazon/) consisting of 1,689,188 reviews. I chose to concentrate on electronics because they tend to have a higher price and presumably people invest some effort to research before they buy them.
 
I focused on highly scoring products (average score 3.5 and up) with more than 50 reviews. I identify two types of reviews:
1. "Dip" reviews: reviews with an exceptional low score (1.5 points lower than that product's average score). 
2. "Sticky" reviews: dip reviews that have an exceptionally low score in a review-window after it.

Thus, "sticky" reviews start a sequence of similarly low score reviews, and cause the average score to decrease temporarily. 



I assume a null hypothesis of no time correlations in the data. The histogram of the shuffled data represents the probability of seeing a given window average score under this hypothesis. 
In the following graph I plot the histogram of the shuffled data in red, and the histogram of the data in green. 
The real data histogram has a lower average and a longer left tail. I conclude that there are time correlations in the data. I defined as "sticky" all reviews with window average score at least 1 point lower than the average score. 
These correspond to the revies at the left tail of the graph, where the probability of encountering such a low score by chance is 20%. The time window here is a window of 5 reviews after the dip review.


###### upload longRev2

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, output_file, show
import numpy as np
import pandas as pd
import pickle
import json
import statistics

''' loading data from files'''       
file0 = open("ElongRev2_beforehist.pkl","rb")
longRev2 = pickle.load(file0)
file0.close()



In [ ]:
len(longRev2)

### making the shuffled data:

In [ ]:
import random
def add_shuff_col(longRev2, comp_name,w):
    add_col = 'shuffled_'+comp_name
    longRev2.loc[:,add_col]  = [0.0]*len(longRev2)
    asin_list = list(longRev2['asin'].unique() )
    list_rand_indices = []
    #temp_list = []
    for name in asin_list:
        indices_of_name = longRev2[longRev2['asin']==name].index.values
        for ind in indices_of_name:
            shuff_wavg = np.mean( longRev2.loc[ np.random.choice( indices_of_name, w ) ,
                                                    comp_name].values )
            #temp_list.append(shuff_wavg)
            #add columns to longRev2: 'shuffled_wavg'
            longRev2.at[ind, add_col]     = float( shuff_wavg )    
    return( longRev2, add_col)

In [ ]:
w=5 #size of time-window
comp_name = 'wavg'# = shuff_name
longRev2, add_col = add_shuff_col(longRev2, comp_name,w)

In [ ]:
''' saving to files'''       
#file10 = open("ElongRev2_with_shuffled_wavg.pkl","wb")
#pickle.dump(longRev2,file10)
#file10.close()


### Plotting the histogram

###### Uploading longRev2 with shuffled window average

In [ ]:

import numpy as np
import pandas as pd
import pickle
import json
import statistics

''' loading data from files'''       
file0 = open("ElongRev2_with_shuffled_wavg.pkl","rb")
longRev2 = pickle.load(file0)
file0.close()


###### Histogram calculaltion

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def hist_comp_col(longRev2, col1, col2, choose_True, Nhistbins):
    df4hist = pd.concat( [longRev2.loc[longRev2[choose_True] == True, col1],
                        longRev2.loc[longRev2[choose_True] == True, col2] ], axis=1, 
                    )
    df4hist.columns = [col1, col2]
    
    #plotting two histograms on the same grid:
    fig, ax = plt.subplots()
    df4hist.plot.kde(ax=ax, legend=False, title='Histogram of values'+choose_True+'=True', bw_method=0.5)
    df4hist.plot.hist(density=True, ax=ax, bins = Nhistbins)
    ax.set_ylabel('Probability')
    ax.grid(axis='y')
    ax.set_facecolor('#d8dcd6')
    print( df4hist.agg(['min', 'max', 'mean', 'std']).round(decimals=2) ) 
    return(df4hist, df4hist.agg(['mean']).round(decimals=2) )


Nhistbins = 30 #numbre of bins in histogram
col1 = comp_name
col2 = add_col
choose_True = 'TFdip'
_, hist_mean = hist_comp_col(longRev2, col1, col2, choose_True, Nhistbins)

###### definition of sticky

Out of 889,708 reviews there are 134,104 reviews that are "dip" reviews, whose score is at least 1.5 points lower than the product's average score. I define as "sticky" as "dip" reviews with window average score at least 1 point lower than the mean window average score (3.70 as calculated above).

In [ ]:
dip_for_sticky = hist_mean.wavg.values[0] - 1
longRev2['TFsticky'] =  longRev2.loc[longRev2['TFdip']==True].wavg< dip_for_sticky


There are 134,104 "dip" reviews
and 10,466 "sticky" reviews

In [ ]:
''' saving to files corrected definition of sticky'''       
#file10 = open("ElongRev2_with_TFsticky.pkl","wb")
#pickle.dump(longRev2,file10)
#file10.close()


## Word cloud of the "sticky" reviews

I continue with calculating the TF-IDF matrices of the sticky versus the dip reviews and pltting them in a word cloud.
For this I use NLTK and Gensim. I use Porter stemmer for stemming. However, because stemmed tokens are not human friendly, I retain the original tokens and use them for presentation in the word cloud.
The word cloud uses weights of the stemmed tokens but choses the most-frequent token that contributed to the stemmed token for presentation.

###### Uploading longRev2 with "TFsticky" review column
TFsticky column: True if stickay, False if dip but not sticky, NaN otherwise'

In [ ]:
import numpy as np
import pandas as pd
import pickle
import json
import statistics

'''upload longRev2 again, corrected definition of sticky'''
file0 = open("ElongRev2_with_TFsticky.pkl","rb")
longRev2 = pickle.load(file0)
file0.close()


###### Word Cloud Calculation

In [ ]:
import sys
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from wordcloud import WordCloud
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

#nltk.download('stopwords')


In [ ]:
def get_common_surface_form(original_corpus, stemmer):
    counts = defaultdict(lambda : defaultdict(int))
    surface_forms = {}

    for document in original_corpus:
        for token in document:
            stemmed = stemmer.stem(token)
            counts[stemmed][token] += 1

    for stemmed, originals in counts.items():
        surface_forms[stemmed] = max(originals, 
                                     key=lambda i: originals[i])

    return surface_forms


In [ ]:
def gen_weights4word_cloud(datadf):
    # expect a pandas series of text
    stemmer = PorterStemmer() # Stemmer for reducing terms to root form 
    stemmed_corpus = []       # For storing the stemmed tokens 
    original_corpus = []      # For storing the non-stemmed tokens


    for rev in datadf:    #iterate over reviews
        contents = rev.lower() # Load file contents
        tokens = word_tokenize(contents)     # Extract tokens
        stemmed = [stemmer.stem(token) for token in tokens] # Stem tokens
        stemmed_corpus.append(stemmed)    # Store stemmed document
        original_corpus.append(tokens)    # Store original document

    dictionary = Dictionary(stemmed_corpus) # Build the dictionary
    # Get the surface form for each stemmed word
    counts = get_common_surface_form(original_corpus, stemmer)
    # Convert to vector corpus
    vectors = [dictionary.doc2bow(text) for text in stemmed_corpus]
    # Train TF-IDF model
    tfidf = TfidfModel(vectors)
    # Get TF-IDF weights (a list of tuples (#, frequency))
    weights = tfidf[vectors[0]]
    # Replace term IDs with human consumable strings
    weights = [(counts[dictionary[pair[0]]], pair[1]) for pair in weights]
    weights = dict(weights)
    return weights


In [ ]:
def gen_word_cloud(weights, wctitle): #expects a dictionary {word:frequency}
    # Initialize the cloud
    wc = WordCloud(
        background_color="white",
        max_words=2000,
        width=1024,
        height=720,
        stopwords=stopwords.words('english')
    )

    # Generate the cloud
    wc.generate_from_frequencies(weights)

    # Save the cloud to a file
    #wc.to_file("word_cloud.png") 

    # Display the generated image:
    plt.imshow(wc, interpolation='bilinear')
    plt.title(wctitle)
    plt.axis("off")
    plt.show()


In [ ]:
sticky_ser = longRev2.reviewText[longRev2.TFsticky==True]
weights_sticky = gen_weights4word_cloud(sticky_ser)

gen_word_cloud(weights_sticky, 'Sticky Word Cloud')
''' saving the tfidf results'''       
file12 = open("Eweights_sticky.pkl","wb")
pickle.dump(weights_sticky,file12)
file12.close()


In [ ]:
#compare sticky reviews to 

dip_ser = longRev2.reviewText[longRev2.TFsticky==False] #includes dip and nondip

weights_dip = gen_weights4word_cloud(dip_ser)

gen_word_cloud(weights_dip,'dip reveiws Word Cloud')

''' saving the tfidf results'''       
file13 = open("Eweights_dip.pkl","wb")
pickle.dump(weights_dip,file13)
file13.close()


## Topic Analysis with LDA: sticky reviews


In 01_Sticky_Reviews_hist_wordCloud
I defined stikcy reviews by the unusuall sequance of low-scorring reviews that follow them.
The data is stored in a dataframe longRev2
Here I use a LDA model to anapyse the topics of the sticky reviews.

A topic is a collection of dominant keywords that are typical to the text. LDA ("Latent Dirichlet Allocation")

In [1]:
import numpy as np
import pandas as pd
import pickle
import statistics

'''upload longRev2 again, corrected definition of sticky'''
file0 = open("../from_TDI190606/capstone_TDI190606/ElongRev2_with_TFsticky.pkl","rb")
longRev2 = pickle.load(file0)
file0.close()



In [3]:
#import sys
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
#from gensim.corpora import Dictionary
#from gensim.models import TfidfModel
#from wordcloud import WordCloud
#from collections import defaultdict
#import matplotlib.pyplot as plt


In [ ]:
import sys
print(sys.executable)

Generally, products have all kind of reviews, some bad, some good. Only part of the bad reviews are sticky reviews, with a sequence of low-score reviews after them. I would like to compare the sticky reviews to the general bad reviews. The Data Frame longRev3 contains only bad reviews - or "dip" reviews as they appear in the code. Here, bad reveiws are defiend by reviews with a score at least 1.5 stars lower than the product's average score.
Sticky reviews, defined in a previous notebook, are labeled "True" in the column "TFsticky".

In [5]:
#longRev3: only dip reviews
longRev3 = longRev2[longRev2.TFdip==True]

In [ ]:
#convert "True" to 1,0
longRev3.TFsticky = longRev3.TFsticky.astype(int)

In [14]:
#topic analysis with  gensim
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
import nltk; 
from nltk.corpus import stopwords

stop_words = stopwords.words('english')


In [13]:
import re
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


Here I choose only the sticky review out of the general bad ("dip") reviews. There are 10466 sticky reviews

In [9]:
# Convert to list
data = longRev3[longRev3.TFsticky==1].reviewText.values.tolist()


In [ ]:

# Remove new line characters
#data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
#data = [re.sub("\'", "", sent) for sent in data]

In [26]:
''' 8. break down each sentence into a list of words through tokenization, 
while clearing up all the messy text in the process.
Gensim’s simple_preprocess is great for this.
'''

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

#data_words is a list of lists of words
data_words = list(sent_to_words(data))

print(data_words[:1][0][:10])


['bought', 'the', 'nook', 'touch', 'instead', 'of', 'the', 'kindle', 'because', 'it']


In [28]:
'''9. Phrases model '''
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=2, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0][0:10]]])


['bought', 'the', 'nook', 'touch', 'instead', 'of', 'the', 'kindle', 'because', 'it']


In [29]:
'''10. Remove Stopwords, Make Bigrams and Lemmatize'''
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [37]:
'''calling the functions above'''
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:10])

['buy', 'nook', 'touch', 'instead', 'kindle', 'sale', 'still', 'alot', 'pay', 'something']


In [38]:
'''11. Create the Dictionary and Corpus needed for Topic Modeling'''
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

#Gensim creates a unique id for each word in the document. 
#The produced corpus shown below is a mapping of (word_id, word_frequency).
# View
#print(corpus[:1])


'Gensim creates a unique id for each word in the document. \nThe produced corpus shown below is a mapping of (word_id, word_frequency).'

In [40]:
''' 12. Train the LDA model. 
We need corpus, dictionary, and decide upon number of topics.
alpha and eta are hyperparameters that affect sparsity of the topics, both defaults to 1.0/num_topics prior.
chunksize is the number of documents to be used in each training chunk. 
update_every determines how often the model parameters should be updated 
and passes is the total number of training passes.
'''
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

...

C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Navit\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

In [41]:
'''13. View the topics in LDA model'''
'''The above LDA model is built with 20 different topics where each topic is a combination of keywords 
and each keyword contributes a certain weightage to the topic.
You can see the keywords for each topic and the weightage(importance) of each keyword 
using lda_model.print_topics() as shown next.
'''

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]



[(0,
  '0.075*"cover" + 0.048*"image" + 0.039*"photo" + 0.033*"idea" + '
  '0.030*"flash" + 0.023*"monitor" + 0.021*"application" + 0.020*"lack" + '
  '0.015*"internal" + 0.013*"unusable"'),
 (1,
  '0.096*"ipad" + 0.058*"show" + 0.038*"add" + 0.035*"contact" + '
  '0.034*"record" + 0.030*"name" + 0.029*"remove" + 0.027*"hd" + 0.024*"color" '
  '+ 0.023*"kind"'),
 (2,
  '0.089*"signal" + 0.055*"audio" + 0.036*"antenna" + 0.036*"range" + '
  '0.035*"live" + 0.024*"house" + 0.023*"pick" + 0.021*"foot" + 0.020*"extend" '
  '+ 0.018*"position"'),
 (3,
  '0.057*"picture" + 0.041*"gb" + 0.039*"watch" + 0.037*"card" + '
  '0.035*"netflix" + 0.030*"movie" + 0.024*"terrible" + 0.022*"browser" + '
  '0.021*"youtube" + 0.019*"disappointed"'),
 (4,
  '0.036*"support" + 0.028*"device" + 0.027*"software" + 0.026*"set" + '
  '0.025*"call" + 0.020*"file" + 0.016*"wifi" + 0.015*"tell" + 0.015*"help" + '
  '0.015*"download"'),
 (5,
  '0.050*"work" + 0.036*"would" + 0.035*"time" + 0.024*"back" + '
  '0.02

In [ ]:
'''14. Compute Model Perplexity and Coherence Score'''
'''Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. 
In my experience, topic coherence score, in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



In [47]:
'''15. Visualize the topics-keywords'''
'''Now that the LDA model is built, the next step is to examine the produced topics and the associated keywords. 
There is no better tool than pyLDAvis package’s interactive chart and is designed to work well with jupyter notebooks.

'''
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.342698  0.085081       1        1  24.872042
5      0.325117  0.040645       2        1  18.653282
14     0.158727  0.111509       3        1   8.153221
4      0.150932 -0.221752       4        1   7.297104
8      0.019435 -0.030053       5        1   5.510801
17     0.022097  0.065714       6        1   4.359994
15     0.002100  0.147911       7        1   3.877184
13    -0.000469 -0.286845       8        1   3.631892
9     -0.007508 -0.248844       9        1   3.160796
18    -0.064056  0.067618      10        1   2.331122
3     -0.074618  0.037919      11        1   2.326899
1     -0.084042  0.026487      12        1   2.157403
6     -0.079827  0.034200      13        1   2.145420
2     -0.091696  0.011436      14        1   1.810947
0     -0.100316  0.030261      15        1   1.773652
19    -0.104637  0.042770      16        1   1.682222
7     -0.094274  0.017282      17        1   1.639673
12    -0.098056  0.010166      18        1   1.628199
11    -0.106396  0.029387      19        1   1.615873
16    -0.115209  0.029108      20        1   1.372266, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
91     Default  11511.000000            not  11511.000000  30.0000  30.0000
949    Default   3442.000000         router   3442.000000  29.0000  29.0000
23     Default   3006.000000         camera   3006.000000  28.0000  28.0000
103    Default   4161.000000        product   4161.000000  27.0000  27.0000
36     Default   3537.000000         device   3537.000000  26.0000  26.0000
147    Default   8323.000000           work   8323.000000  25.0000  25.0000
806    Default   2220.000000         window   2220.000000  24.0000  24.0000
57     Default   8179.000000            get   8179.000000  23.0000  23.0000
40     Default   6070.000000             do   6070.000000  22.0000  22.0000
21     Default   4167.000000            buy   4167.000000  21.0000  21.0000
131    Default   4349.000000           time   4349.000000  20.0000  20.0000
115    Default   1792.000000         screen   1792.000000  19.0000  19.0000
113    Default   2652.000000         return   2652.000000  18.0000  18.0000
2069   Default   2016.000000         tablet   2016.000000  17.0000  17.0000
148    Default   4854.000000          would   4854.000000  16.0000  16.0000
17     Default   4484.000000             be   4484.000000  15.0000  15.0000
140    Default   7495.000000            use   7495.000000  14.0000  14.0000
419    Default   1872.000000          phone   1872.000000  13.0000  13.0000
29     Default   1902.000000       computer   1902.000000  12.0000  12.0000
4886   Default   1370.000000           ipad   1370.000000  11.0000  11.0000
139    Default   1582.000000            usb   1582.000000  10.0000  10.0000
22     Default   1554.000000          cable   1554.000000   9.0000   9.0000
1526   Default   1705.000000             tv   1705.000000   8.0000   8.0000
31     Default   1825.000000        connect   1825.000000   7.0000   7.0000
2427   Default   1693.000000            app   1693.000000   6.0000   6.0000
254    Default   2787.000000        problem   2787.000000   5.0000   5.0000
356    Default   1413.000000          sound   1413.000000   4.0000   4.0000
220    Default   1728.000000        support   1728.000000   3.0000   3.0000
14     Default   3113.000000           back   3113.000000   2.0000   2.0000
25     Default   1193.000000         charge   1193.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
1042   Topic20    692.095032           full    692.875671   4.2876  -2.5754
342    Topic20    522.427979        adapter    523.208618   4.2872  -2.8566
1391   Topic20    284.889343           lock    285.669983   4.2860  -3.4630
1061   Topic20    219.998413     

In [ ]:
'''Each bubble on the left-hand side plot represents a topic. 
The larger the bubble, the more prevalent is that topic.
A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart 
instead of being clustered in one quadrant.
A model with too many topics, will typically have many overlaps, 
small sized bubbles clustered in one region of the chart.
If you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. 
These words are the salient keywords that form the selected topic.

we will focus on how to arrive at the optimal number of topics given any large corpus of text.'''